### Algoritmo KNN

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn import metrics, preprocessing
warnings.filterwarnings(action='ignore')                  # Desactivar las advertencias.
%matplotlib inline

La explicación sobre el conjunto de datos se puede encontrar [aquí](https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones).

In [4]:
df.shape # dimesiones del dataset

(19622, 160)

In [5]:
df.columns

Index(['Unnamed: 0', 'user_name', 'raw_timestamp_part_1',
       'raw_timestamp_part_2', 'cvtd_timestamp', 'new_window', 'num_window',
       'roll_belt', 'pitch_belt', 'yaw_belt',
       ...
       'gyros_forearm_x', 'gyros_forearm_y', 'gyros_forearm_z',
       'accel_forearm_x', 'accel_forearm_y', 'accel_forearm_z',
       'magnet_forearm_x', 'magnet_forearm_y', 'magnet_forearm_z', 'classe'],
      dtype='object', length=160)

## Análisis EDA

In [6]:
 df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19622 entries, 0 to 19621
Columns: 160 entries, Unnamed: 0 to classe
dtypes: float64(94), int64(29), object(37)
memory usage: 24.0+ MB


In [7]:
df.dtypes # muestra la categría de los datos y la variable a la que pertenecen

Unnamed: 0                int64
user_name                object
raw_timestamp_part_1      int64
raw_timestamp_part_2      int64
cvtd_timestamp           object
                         ...   
accel_forearm_z           int64
magnet_forearm_x          int64
magnet_forearm_y        float64
magnet_forearm_z        float64
classe                   object
Length: 160, dtype: object

In [8]:
df.head(2)  # lee las primera dos filas

,Unnamed: 0,user_name,raw_timestamp_part_1,raw_timestamp_part_2,cvtd_timestamp,new_window,num_window,roll_belt,pitch_belt,yaw_belt,...,gyros_forearm_x,gyros_forearm_y,gyros_forearm_z,accel_forearm_x,accel_forearm_y,accel_forearm_z,magnet_forearm_x,magnet_forearm_y,magnet_forearm_z,classe
0,1,carlitos,1323084231,788290,05/12/2011 11:23,no,11,1.41,8.07,-94.4,...,0.03,0.0,-0.02,192,203,-215,-17,654.0,476.0,A
1,2,carlitos,1323084231,808298,05/12/2011 11:23,no,11,1.41,8.07,-94.4,...,0.02,0.0,-0.02,192,203,-216,-18,661.0,473.0,A


In [9]:
df.isnull().values.any()  #devuelve True si hay, al menos, un valor faltante. En este caso hay valores faltantes.

True

In [10]:
df.isnull().sum().sum() # devuelve el número de valores faltantes

1921600

Elimino las columnas que tengan más del 97% de valores perdidos.

In [11]:
df.dropna(axis=1, thresh=19033, inplace=True) # 19033 es el 97% del total de registros
print(df.shape)
# inplace=True actualiza el cambio que se ha hecho al eliminar las columnas. Cuando imprimo la tabla nuevamete el número 
# de columnas pasa de 160 a 60.       # <Tu código va aquí>

(19622, 60)


Elimino aquellas variables que no aportan al modelo.

In [12]:
df = df.drop(['Unnamed: 0', 'user_name','raw_timestamp_part_1', 'raw_timestamp_part_2', 'cvtd_timestamp','new_window'], axis=1)
print(df.shape)
# el número de columnas pasa de 60 a 54.

(19622, 54)


Preprocesamiento de la variable dependiente con LabelEncoder

In [13]:
pre = preprocessing.LabelEncoder()
df['classe'] = pre.fit_transform(df['classe'])
df['classe'].unique()

array([0, 1, 2, 3, 4])

Preprocesamiento de los datos con MinMaxScaler

In [14]:
X = df.drop(columns = ['classe'])
y = df['classe']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

pre = preprocessing.MinMaxScaler()
X_train = pre.fit_transform(X_train)
X_test = pre.fit_transform(X_test)

In [15]:
# Tabla de salida

cols = ['Case','KNN']           

df = pd.DataFrame(columns=cols)
df.set_index("Case",inplace=True)
df.loc['Standard'] = [0]


In [16]:
# ALGORITMO A UTILIZAR KNN
knn   = KNeighborsClassifier()

models = [knn]

col = 0
for model in models:
    model.fit(X_train,y_train.values.ravel())
    df.iloc[0,col] = model.score(X_test,y_test)
    col += 1

df.head()

# Hiperparámetro predeterminado = 0.95. Puede que otros hiperparámetros como GridSearchCV o Hyperopt (hiperoptimización 
# bayesiana) consiga aumentar algunos puntos del hiperparámetro predeterminado (Standard).

,KNN
Case,
Standard,0.955157


In [20]:
# CALCULAR LA PRECISION DEL MODELO CON EL ALGORITMO KNN
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors=3).fit(X_train, y_train)
y_hat = knn.predict(X_test)

print('Accuracy：', accuracy_score(y_test, y_hat))

Accuracy： 0.9671830411740726


accuracy > 90%

EL accuracy obtenido es de 0.967, por lo tanto, el modelo acierta el 96.7% de las veces.